## Dependencies

---

In [34]:
import yaml
import re
from datetime import datetime
import pandas as pd

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException


Setting up web driver.

In [36]:
EdgeOptions = Options()
EdgeOptions.add_argument("--headless")
edge = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()), options=EdgeOptions)
# edge = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))


## Extract

---

### Functions:

In [37]:
def resources(name: str) -> dict[str:str]:
    """Read yaml file to get xpath resources
    
    :param name: name of the resource to extract
    :return: the xpath
    """
    with open("xpath.yml", "r") as f:
        content: dict[str:str] = yaml.safe_load(f)
        return content[name]

In [38]:
def close_prompt(driver: webdriver) -> None:
    """Close prompt when is required
    
    :param driver: browser instance 
    """
    button: str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [39]:
def click_element(element: webdriver, path: str) -> None:
    """Click on element in the current browser page
    
    :param element: browser instance
    :param path: xpath of element to click
    """
    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [40]:
def extract_feature_in(element: webdriver, path: str) -> str:
    """Extract ad feature where current browser page is the ad page
    
    :param element: browser instance
    :param path: xpath of feature to extract
    :return: feature extracted
    """
    feature: str = element.find_element(by=By.XPATH, value=path)
    return feature

In [41]:
def extract_feature_out(element: webdriver, path: str) -> list[str]:
    """Extract ad feature where current browser page is the listing page (not in the ad)
    
    :param element: browser instance
    :param path: xpath of feature to extract
    :return: list of features
    """
    feature = []

    try:
        feature: str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")

In [42]:
def persistence(dataframe: pd.DataFrame) -> None:
    """Save dataframe to csv file
    
    :param dataframe: dataframe to save
    """
    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [43]:
def get_len(features: list[list[str]]) -> bool:
    """Check if all features has the same length to create new df
    
    :param features: list of features extracted with scraping
    :return: a list with all lengths where first place is the len of the first element and the mostly accepted.
    """
    base_len = len(features[0])
    lengths = []
    lengths.append(base_len)
    for feature in features:
        if len(feature) == base_len:
            continue
        else:
            lengths.append(len(feature))
    return lengths

In [44]:
def check_len(lengths: list[int]) -> bool:
    """Check if all elements in list has the same value.
    
    :param lengths: list of lengths
    :return: a boolean
    """
    for length in lengths[1:]:
        if length != lengths[0]:
            return False
        return True

Variables pool.

In [45]:
descriptions: list[str] = []
prices: list[int] = []
models: list[str] = []
years: list[int] = []
kilometers: list[int] = []
engines: list[str] = []

# seconds
DELAY: int = 1
SEDAN = resources("cars_sedan")

Get web page and close prompt (when is required).

In [46]:
edge.get(SEDAN["url"])
# close_prompt(edge)

Extract car features.

In [47]:
# Get this list of ads and click on the first one
elements = edge.find_elements(by=By.XPATH, value=SEDAN["ad"])
elements[0].click()

try:
    for element in range(len(elements)):
        # Wait until the element is present to avoid missing data
        dynamic_delay = WebDriverWait(edge, DELAY).until(
            EC.presence_of_element_located((By.XPATH, SEDAN["description"])))

        brand_str = extract_feature_in(edge, SEDAN["brand"]).text
        model_str = extract_feature_in(edge, SEDAN["model"]).text
        models.append(brand_str + " " + model_str)

        descriptions.append(extract_feature_in(edge, SEDAN["description"]).text)
        prices.append(extract_feature_in(edge, SEDAN["price"]).text)
        kilometers.append(extract_feature_in(edge, SEDAN["kilometers"]).text)
        engines.append(extract_feature_in(edge, SEDAN["engine"]).text)
        years.append(extract_feature_in(edge, SEDAN["year"]).text)

        click_element(edge, SEDAN["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, SEDAN["exit"])

## Export

---

### Functions:

In [48]:
def parse_price(dataframe: pd.DataFrame, feature: str, pattern: str) -> None:
    """Parse financial features like prices. Works with $#,### (not dot)
    
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param pattern: feature pattern to transform
    """
    dataframe[feature] = dataframe[feature] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",", "")))


In [49]:
def remove_chars(dataframe: pd.DataFrame, feature: str, wildcards: str) -> pd.Series:
    """Remove escaped characters from features like indented large extracts of text
    
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param wildcards: escaped characters to remove
    :return: Serie without escaped characters
    """
    return (dataframe
    .apply(
        lambda row: row[feature]
        , axis=1
    )
    .apply(
        lambda extracted_f: re.sub(wildcards, r' ', extracted_f)
    )
    )


In [50]:
PRICE_RG: str = r"^\$(?P<prices>\d+,?\d+)?"
KM_REG: str = r"(?P<km>^\d+)"
CHARACTER_RG: str = '(\n|\r)+'

Check that all features has the same length.

In [51]:
lengths = [len(engines), len(models), len(years), len(kilometers), len(prices), len(descriptions)]
lengths

[50, 50, 50, 50, 50, 50]

Create dataframe to transform data extracted

In [56]:
if check_len(lengths):
    df_temp = pd.DataFrame({
        "model": models,
        "year": years,
        "kilometers": kilometers,
        "engine": engines,
        "price": prices,
        "description": descriptions
    })

    # Parse features. String to number and raw string with characters to structured string.
    parse_price(df_temp, "price", PRICE_RG)
    df_temp["kilometers"] = df_temp["kilometers"].str.extract(KM_REG).applymap(int)
    df_temp["year"] = df_temp["year"].map(int)
    df_temp["description"] = remove_chars(df_temp, "description", CHARACTER_RG)

    persistence(df_temp)


In [57]:
df_temp

,model,year,kilometers,engine,price,description
0,Honda Civic,2019,46771,Gasolina,13950,Garantía: Como es visto no hay garantía Financ...
1,Nissan Sentra,2016,95600,Gasolina,8000,Precio negociable: si Color : Azul DESCRIPCIÓN...
2,otros + Datsun 120 Y 1979,1979,0,Gasolina,3900,Precio negociable: si Color : Verde Vendo boni...
3,Kia SOUL,2015,86000,Gasolina,8800,Garantía: Como es visto no hay garantía Financ...
4,Honda Civic,2016,109500,Gasolina,13000,Color : Gris DESCRIPCIÓN CALIFICACIONES SEGURI...
5,Nissan Sentra,2019,68,Gasolina,9300,recién ingresado motor 1.8 bolsas todas buenas...
6,Hyundai Elantra,2016,84,Gasolina,10000,Garantía: Como es visto no hay garantía Color ...
7,Hyundai Elantra,2018,47,Gasolina,12500,Color : Blanco Se vende un 2018 Hyundai Elantr...
8,Kia Rio,2013,125300,Gasolina,6200,Financiamiento: no Precio negociable: si Color...
9,Nissan Sentra,2024,131,Gasolina,6400,Precio negociable: si Color : Blanco A toda Pr...
